In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

def read_rtp_file(file_path):
    with open(file_path, 'r') as file:
        rtp_data = file.readlines()  # Read the file as a list of lines
    return rtp_data
def parse_rtp(rtp_data):
    rtp_dict = {}
    residue_type = None
    atom_dict = {}
    
    # Process each line in the RTP data
    for line in rtp_data:
        line = line.strip()
        
        # If a new residue type is found, initialize a new dictionary for it
        residue_match = re.match(r"\[\s*[A-Z0-9]*\s*\]", line)
        if residue_match:
            if residue_type:  # If there is an existing residue, save it before moving on
                rtp_dict[residue_type] = atom_dict
            
            residue_type = residue_match.group(0)[2:-2]
            atom_dict = {}
            reading_atoms = True  # Start reading atoms
        # Process atoms section (inside [ atoms ] block)
        elif residue_type and re.match(r"\[ atoms \]", line):
            # Start reading atoms after "[ atoms ]" section header
            continue
        elif re.match(r"\[ bonds \]", line) or re.match(r"\[ impropers \]", line) or re.match(r"\[ cmap \]", line) or re.match(r"\[ bondedtypes \]", line):
            # Stop reading when the "[ bonds ]" section is reached
            reading_atoms = False
            continue
        elif residue_type and reading_atoms:
            # Parse atom data line
            parts = line.split()
            if len(parts) >= 2:
                atom_name = parts[0]
                atom_type = parts[1]
                if atom_name.isupper() and atom_type.isupper():
                    atom_dict[atom_type] = atom_name

    # Don't forget to add the last residue type to the dictionary
    if residue_type:
        rtp_dict[residue_type] = atom_dict
    
    return rtp_dict

def get_atom_data(residue, atom, rtp_path='forcefield/aminoacids.rtp', nb_path='forcefield/ffnonbonded.csv'):
    rtp_lines = read_rtp_file(rtp_path)
    rtp_data = parse_rtp(rtp_lines)
    nb_data = pd.read_csv(nb_path, sep='\t', comment=';')
    atom_type = rtp_data[residue][atom]
    row =  nb_data[nb_data['name'] == atom_type]
    return row['mass'].values[0], row['sigma'].values[0], row['epsilon'].values[0]

get_atom_data('LYS', 'NH1')


(14.007, 0.329632525712, 0.8368)

In [2]:
import mdtraj as md
import nglview as nv

In [17]:
traj = md.load('outputs/GTT_test/sc2_ckp-250_noise-0.003/frame_0_dt.pdb')
traj.center_coordinates()
view = nv.show_mdtraj(traj[::-1])
view.clear()
view.add_ball_and_stick()
view

NGLWidget(max_frame=99)

In [19]:
import numpy as np
np.load('energy_files/energies_GTT_test_sc2_ckp-250.npy')

array([-1307.55921457, -1489.0187856 ,   371.67451982, -1779.75834299])

In [2]:
from memory_profiler import profile
@profile
def load_pickle():
    import pickle
    with open("train_features/feats_training.pkl", "rb") as f:
        topologies = pickle.load(f)
    return topologies

data = load_pickle()
data.keys()

ERROR: Could not find file /scratch/local/jobs/30187308/ipykernel_278729/1182152713.py


dict_keys(['res', 'atom', 'xyz', 'mask', 'map', 'esm_emb'])